# MCP Agent with Microsoft Learn Documentation

This notebook demonstrates how to create an AI agent that can search and fetch Microsoft Learn documentation using the Model Context Protocol (MCP).

## Import the Needed Packages

We'll import all the necessary libraries for creating our MCP-enabled agent.

In [1]:
import os
import asyncio
import json
from dotenv import load_dotenv

# Add references
from azure.identity import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

from IPython.display import display, HTML

## Setting up Azure OpenAI Connection

Configure the Azure OpenAI service using Azure AD authentication.

In [2]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# Create Azure credential 
credential = DefaultAzureCredential()

# Create a token provider function
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI."""
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()

## Configure MCP Plugin

Set up the Model Context Protocol plugin to connect to Microsoft Learn documentation.

In [3]:
# MCP server configuration
mcp_server_url = "https://learn.microsoft.com/api/mcp"
mcp_server_label = "mslearn"

# Create MCP Plugin for Semantic Kernel
mcp_plugin = MCPStreamableHttpPlugin(
    name=mcp_server_label,
    url=mcp_server_url
)

# Create kernel
kernel = Kernel()

print(f"MCP Plugin configured for: {mcp_server_label} at {mcp_server_url}")

MCP Plugin configured for: mslearn at https://learn.microsoft.com/api/mcp


## Creating the MCP Agent

Create an AI agent with access to Microsoft Learn documentation through MCP tools.

In [4]:
# Create agent with MCP plugin via kernel
agent = ChatCompletionAgent(
    service=chat_completion_service,
    kernel=kernel,
    name="Microsoft-Learn-Agent",
    instructions="""
    You are a helpful assistant with access to Microsoft Learn documentation through MCP tools.
    
    ## Querying Microsoft Documentation
    
    You have access to MCP tools called `microsoft_docs_search` and `microsoft_docs_fetch` - these tools allow you to 
    search through and fetch Microsoft's latest official documentation, and that information might be more detailed 
    or newer than what's in your training data set.

    When handling questions around how to work with native Microsoft technologies, such as C#, F#, ASP.NET Core, 
    Microsoft.Extensions, NuGet, Entity Framework, the `dotnet` runtime, Azure services, or any Microsoft 
    technologies - please use these tools for research purposes when dealing with specific / narrowly defined 
    questions that may occur.
    
    ALWAYS use the available MCP tools to search Microsoft Learn documentation when users ask about:
    - Azure services and configuration
    - Microsoft technologies and frameworks
    - Official documentation and best practices
    - Step-by-step guides and tutorials
    
    Use phrases like "searching Microsoft docs" to indicate you're using the tools.
    """
)

print(f"Created agent: {agent.name}")

Created agent: Microsoft-Learn-Agent


## Initialize MCP Connection

Connect to the MCP server and load available tools and prompts.

In [5]:
async def initialize_mcp():
    """Initialize MCP plugin connection and load tools."""
    try:
        await mcp_plugin.connect()
        print(f"MCP Plugin connected successfully")
        
        # Load tools and prompts
        await mcp_plugin.load_tools()
        await mcp_plugin.load_prompts()
        print("MCP tools and prompts loaded")
        
        # Add the MCP plugin to the kernel
        kernel.add_plugin(mcp_plugin)
        print(f"MCP plugin added to kernel")
        
        return True
        
    except Exception as e:
        print(f"MCP Plugin setup error: {e}")
        import traceback
        traceback.print_exc()
        return False

# Initialize the MCP connection
mcp_initialized = await initialize_mcp()
print(f"\nMCP initialization {'successful' if mcp_initialized else 'failed'}")

MCP Plugin connected successfully
MCP tools and prompts loaded
MCP plugin added to kernel

MCP initialization successful


## Agent Interaction Function

Create a function to handle user interactions with the agent, displaying both the conversation and function calls.

In [6]:
async def chat_with_agent(user_input, thread=None):
    """Chat with the agent and display results with function call details."""
    
    html_output = (
        f"<div style='margin-bottom:10px'>"
        f"<div style='font-weight:bold'>User:</div>"
        f"<div style='margin-left:20px'>{user_input}</div></div>"
    )

    agent_name = None
    full_response: list[str] = []
    function_calls: list[str] = []

    # Buffer to reconstruct streaming function call
    current_function_name = None
    argument_buffer = ""

    try:
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # Display function calls if any were made
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>MCP Tool Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))
        return thread
        
    except Exception as e:
        print(f"Error during agent execution: {e}")
        return thread

print("Agent interaction function ready!")

Agent interaction function ready!


## Test the Agent

Now let's test our MCP-enabled agent with some questions about Microsoft technologies and Azure services.

In [7]:
# Test questions for the agent
test_questions = [
    #"What are the best practices for Azure Functions development?",
    #"How do I configure Azure App Service authentication?",
    "What's the difference between Azure Storage accounts types?"
]

# Run the conversation
thread = None
for question in test_questions:
    thread = await chat_with_agent(question, thread)
    print("\n" + "="*80 + "\n")  # Separator between questions

## Interactive Chat

Use this cell to ask your own questions to the agent. The agent will search Microsoft Learn documentation to provide accurate, up-to-date answers.

In [9]:
# Interactive chat - modify the question below
user_question = "How do I deploy a .NET application to Azure App Service?"

# Use the existing thread to maintain conversation context
thread = await chat_with_agent(user_question, thread)

## Cleanup

Clean up the MCP connection when done.

In [8]:
# Clean up MCP connection
try:
    await mcp_plugin.close()
    print("MCP Plugin closed successfully")
except Exception as e:
    print(f"Error closing MCP plugin: {e}")

Error closing MCP plugin: Attempted to exit cancel scope in a different task than it was entered in
